In [68]:

import yfinance as yf
import pandas as pd
import numpy as np



In [62]:
import yfinance as yf
import pandas as pd

tickers = ['^FTSE', '^GDAXI', 'FTSEMIB.MI', '^AEX', '^FCHI', '^IBEX']
index_to_country = {'^FTSE': 'UK', '^GDAXI': 'Germany', 'FTSEMIB.MI': 
                    'Italy', '^AEX': 'Netherlands', '^FCHI': 'France', '^IBEX': 'Spain'}

start_date = '2004-01-01'
end_date = '2024-01-01'

dfs = []
for index in tickers:
    data = yf.download(index, start=start_date, end=end_date)
    data['Index'] = index
    dfs.append(data)
index_data = pd.concat(dfs)
index_data['Country'] = index_data['Index'].map(index_to_country)
index_data.head(10)

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Index,Country
Date,,,,,,,,
2004-01-02,4476.899902,4518.000000,4476.899902,4510.200195,4510.200195,578018400,^FTSE,UK
2004-01-05,4510.200195,4515.200195,4494.899902,4513.299805,4513.299805,1405558400,^FTSE,UK
2004-01-06,4513.299805,4522.899902,4489.899902,4505.200195,4505.200195,2059454200,^FTSE,UK
2004-01-07,4505.200195,4512.100098,4466.200195,4473.000000,4473.000000,2138137700,^FTSE,UK
2004-01-08,4473.000000,4520.100098,4473.000000,4494.200195,4494.200195,2147189600,^FTSE,UK
2004-01-09,4494.200195,4495.799805,4445.100098,4466.299805,4466.299805,2147016700,^FTSE,UK
2004-01-12,4466.299805,4466.299805,4443.399902,4449.600098,4449.600098,1758017200,^FTSE,UK
2004-01-13,4449.600098,4477.899902,4436.600098,4440.100098,4440.100098,1921308400,^FTSE,UK
2004-01-14,4440.100098,4464.500000,4431.100098,4461.399902,4461.399902,2142098400,^FTSE,UK


In [67]:
import pandas as pd
election_data = 'C:\\Users\\rafme\\Downloads\\Election_data.xlsx'
df =pd.read_excel(election_data)
print(df.head())

   Unnamed: 0           Unnamed: 1                            Unnamed: 2  \
0         NaN            Elections                                   NaN   
1         NaN              Germany  Bundestagswahlen (Federal elections)   
2         NaN                 Date                                Winner   
3         NaN  2021-09-26 00:00:00                                Scholz   
4         NaN  2017-09-24 00:00:00                                Merkel   

  Unnamed: 3   Unnamed: 4 Unnamed: 5  
0        NaN          NaN        NaN  
1        NaN          NaN        NaN  
2      Party     Spectrum      Share  
3        SPD  centre-left       25.7  
4        CDU       centre       32.9  


In [69]:
def calculate_max_profit(df):
    ''' 
    Calculate the maximum profit that can be made by buying and selling a stock.

    Parameters:
    - df: DataFrame containing historical stock data with 'Close' prices.

    Returns:
    - max_profit: Maximum profit that can be made.
    - buy_date: Date to buy the stock for maximum profit.
    - sell_date: Date to sell the stock for maximum profit'''
    max_profit = 0
    buy_date = None
    sell_date = None
    for i in range(len(df) - 1):
        for j in range(i + 1, len(df)):
            profit = (df['Close'].iloc[j] - df['Close'].iloc[i])/df['Close'].iloc[i]
            if profit > max_profit:
                max_profit = profit
                buy_date = df.index[i]
                sell_date = df.index[j]

    return max_profit, buy_date, sell_date

for index in tickers:
    subset = index_data[index_data['Index'] == index]
    max_profit, buy_date, sell_date = calculate_max_profit(subset)
    print(f"Index: {index}, Max Profit: {max_profit*100:.2f}%, Buy Date: {buy_date}, Sell Date: {sell_date} ")


Index: ^FTSE, Max Profit: 128.19%, Buy Date: 2009-03-03 00:00:00, Sell Date: 2023-02-20 00:00:00 


KeyboardInterrupt: 